<!--NAVIGATION-->
<a href="https://colab.research.google.com/github/marcoteran/deeplearningmodule/blob/main/01_unsupervisedlearning_kmeans/01_unsupervisedlearning_kmeans.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir en Colab" title="Abrir y ejecutar en Google Colaboratory"></a>

### Instalar dependencias para Colab y carga del dataset
* Ejecutar la siguiente dependencia para instlar mlutils
* Descargar el dataset en la carpeta ``data``

In [ ]:
!pip install mlutils

In [ ]:
!rm -rf data/
!rm -rf data.z*
!drm -rf mlutils.p*
!mkdir -p data/
!wget https://raw.githubusercontent.com/marcoteran/deeplearningmodule/main/01_unsupervisedlearning_kmeans/mlutils.py
!wget https://github.com/marcoteran/deeplearningmodule/raw/main/01_unsupervisedlearning_kmeans/data.zip
!unzip data.zip
!ls

## Ejemplos de código
# Aprendizaje no supervisado: Análisis de grupos con k-means

*Name:* Marco Teran
*E-mail:* marco.tulio.teran@gmail.com,
[Website](http://marcoteran.github.io/),
[Github](https://github.com/marcoteran),
[LinkedIn](https://www.linkedin.com/in/marcoteran/).

### Importar librerías importantes

Empezamos con las importaciones estándar:

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import mlutils
import numpy as np
import pandas as pd
import importlib
importlib.reload(mlutils)
#reload(mlutils) Python2

# Visualizations
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()  # for plot styling

# Machine learning
from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering, DBSCAN
from IPython.display import HTML
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from scipy.stats import mode
from sklearn.metrics import pairwise_distances_argmin

from IPython.display import Image
%matplotlib inline

# ¿Qué es clustering?

- **Objetivo**: agrupar objetos físicos o obstractos en clases de objetos **similares**
- Es una tarea **NO SUPERVISADA** $\rightarrow$ no sabemos a priorí cómo clasificar nuestros objetos
- Es una tarea **NO COMPLETAMENTE DEFINIDA** $\rightarrow$ ¿Cómo cuantificamos el desempeño de un resultado de clustering?

- ¿Qué definición de **similitud** establecemos?

**Ejemplos de aplicaciones de clustering**

- Taxonomías en biología, agrupaciones por similitud biológica, o incluso genética (big data!)
- Páginas similares para estructurar resultados de búsquedas (p.ej. La búsqueda de "película" podría devolver resultados agrupados por descripciones similares.
- Segmentación de clientes o usuarios por un criterio de similitud definido.


# Ejemplo 1: Agrupar objetos por semejanza con k-means

En las secciones anteriores, hemos explorado una categoría de modelos de aprendizaje de máquinas sin supervisión: la reducción de la dimensionalidad.
Aquí pasaremos a otra clase de modelos de aprendizaje automático no supervisado: los algoritmos de agrupamiento.
Los algoritmos de agrupación buscan aprender, a partir de las propiedades de los datos, una división óptima o un etiquetado discreto de grupos de puntos.

Muchos algoritmos de clustering están disponibles en Scikit-Learn y en otros lugares, pero quizás el más simple de entender es un algoritmo conocido como *k-means clustering*, que se implementa en ``sklearn.cluster.KMeans``.

El ejemplo se realizará con un conjunto sencilloo de datos bidimensionales

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=150, 
                  n_features=2, 
                  centers=3, 
                  cluster_std=0.5, 
                  shuffle=True, 
                  random_state=0)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(X[:, 0], X[:, 1], 
            c='white', marker='o', edgecolor='black', s=50)
plt.grid()
plt.tight_layout()
#plt.savefig('images/11_01.png', dpi=300)
plt.show()

El conjunto de datos consiste en **150 puntos** generados de forma aleatoria y agrupados en **tres regiones** con alta densidad, que se visualizan mediante un diagrama de dispersión bidmensional

* En las aplicaciones de agrupamiento reales no tenemos ninguna información de la categoría verdadera básica sobre las muestras
* Nuestro objetivo es agrupar las muestras basandonos en sus semejanzas de caracteristicas

**Algoritmo de k-means:**
1. Selecciona aleatoriamente k centroides a partir de los puntos de muestra como centros de grupos iniciales
2. Asigna a cada muestra al centroide más cercano $\mu^{(j)},\,j\in\{1,\ldots,k\}$
3. Desplaza los centroides al centro de las muestras asignadas para ello
4. Repite los pasos 2 y 3 hasta que las asignaciones de grupos no cambien o hasta conseguir una tolerancia definidida por el usuario o el número máximo de iteraciones

**Pseudo código**

    Entrada: 
        X: datos
        k: número de clusters deseados
        
    Algoritmo:
        1. Repite hasta que los k centroides no cambien:
        2. Selecciona k centroides aleatoriamente
        3. Establece k clusters asignado cada dato al centroide más cercano
        4. Recalcula el centroide de cada cluster como el promedio de los datos

* La semejanza entre objetos se mide como el opuesto a la distancia entre estos
* Una distancia utilizada habitualmente para las muestras de agrupamiento con caracteristicas continuas es la distancia euclidiana al cuadrado entre dos puntos x y y en un espacio m-dimensional
$$d(x,y)^2=\sum_{j=1}^{m}{(x_j-y_j)^2}=||x-y||_2^2$$

* En la ecuación anterior el indice $j$ se refiera a la $j^a$ dimensión (columna de carateristicas) de los puntos de muestra $x$ y $y$
* Los superindices $i$ y $j$ se utilizaran para hacer referencia al índice de muestra y al indice de grupo respectivamente

El algoritmo k-meas en un algoritmo de optimización que consiste en un método iterativo para minimizar la *Suma de errores cuadraticos (SSE)* dentro del grupo (inercia del grupo)

$$ SSE=\sum_{i=1}^{n}{\sum_{j=1}^{k}{w^{(i,j)} ||x^{(i)}-\mu^{(j)}||_2^2}}$$

El algoritmo se implementa en ``scikit-learn`` mediante la clase ``KMeans`` del módulo ``cluster``

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
km = KMeans(n_clusters=3, 
            init='random', 
            n_init=10, 
            max_iter=300,
            tol=1e-04,
            random_state=0)
y_km = km.fit_predict(X)

* Se establece el numero de grupos deseados en $3$
* La especificación del número de grupos es una de las limitaciones de k-means
* ``n_init=10`` se especifica para ejecutar el algoritmo de agrupamiento k-menas 10 veces independiemente con centroidos aleatorios para elegir el modelo final con el SSE más bajo
* ``max_iter=300`` especifica el numero de iteraciones máxima para cada ejecución única
* La implementación de k-means se detiene si converge antes de llegar al numero máximo iteraciones
* ``tol`` es el parametro de tolerancia que controla los cambios en la suma de errores cuadraticos dentro del grupo de convergencia: entre más alto, menos sensibilidad ala convergencia y más rápidos

* Uno o más grupos pueden quedar vacios
* A estos grupos son asignados las muesras ubicadas en los puntos más lejanos
* Es necesario que los datos reales utlicen la misma metrica, es decir, se encuentren en la misma escala: Aplicar estandarización

Despues de predecir las etiquetas $y_{km}$, podemos verificar los grupos que k-means ha identidicado en el conjunto de datos junto a los centroides de los grupos

Estos se encuentran almacenados en el atributo cluster_centrers_ del objeto KMeans ajustado

In [ ]:
plt.scatter(X[y_km == 0, 0],
            X[y_km == 0, 1],
            s=50, c='lightgreen',
            marker='s', edgecolor='black',
            label='cluster 1')
plt.scatter(X[y_km == 1, 0],
            X[y_km == 1, 1],
            s=50, c='orange',
            marker='o', edgecolor='black',
            label='cluster 2')
plt.scatter(X[y_km == 2, 0],
            X[y_km == 2, 1],
            s=50, c='lightblue',
            marker='v', edgecolor='black',
            label='cluster 3')
plt.scatter(km.cluster_centers_[:, 0],
            km.cluster_centers_[:, 1],
            s=250, marker='*',
            c='red', edgecolor='black',
            label='centroides')
plt.legend(scatterpoints=1)
plt.grid()
plt.tight_layout()
#plt.savefig('images/11_02.png', dpi=300)
plt.show()

Diagrama de dispersión: k-means ha situado los centroides en el centro de cada esfera

## Método del codo *elbow* para encontrar el número optimo de grupos

In [ ]:
print('Inercia del cluster: %.2f' % km.inertia_)

In [ ]:
distortions = []
for i in range(1, 11):
    km = KMeans(n_clusters=i, 
                init='k-means++', 
                n_init=10, 
                max_iter=300, 
                random_state=0)
    km.fit(X)
    distortions.append(km.inertia_)
plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Número de clusters')
plt.ylabel('Inercia del cluster')
plt.tight_layout()
#plt.savefig('./figures/elbow.png', dpi=300)
plt.show()

Dela gráfica anterior se peude aprecuas que para k=3 hay una buena opción de agrupamiento de datos

## Cuantificar la calidad del agrupamiento mediante gráficos de silueta

En el siguiente gráfico generamos los coeficientes de silueta para un agrupamiento de k-means con k=3

In [ ]:
import numpy as np
from matplotlib import cm
from sklearn.metrics import silhouette_samples

km = KMeans(n_clusters=3, 
            init='k-means++', 
            n_init=10, 
            max_iter=300,
            tol=1e-04,
            random_state=0)
y_km = km.fit_predict(X)

cluster_labels = np.unique(y_km)
n_clusters = cluster_labels.shape[0]
silhouette_vals = silhouette_samples(X, y_km, metric='euclidean')
y_ax_lower, y_ax_upper = 0, 0
yticks = []
for i, c in enumerate(cluster_labels):
    c_silhouette_vals = silhouette_vals[y_km == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(float(i) / n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
             edgecolor='none', color=color)

    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(c_silhouette_vals)
    
silhouette_avg = np.mean(silhouette_vals)
plt.axvline(silhouette_avg, color="red", linestyle="--") 

plt.yticks(yticks, cluster_labels + 1)
plt.ylabel('Cluster')
plt.xlabel('Coeficiente de silueta')

plt.tight_layout()
# plt.savefig('./figures/silhouette.png', dpi=300)
plt.show()

A partir de la grafica anterior se pueden obervar los tamaños de los dieferentes grupos e identidicar aquellso que continuen outlieres

La línea punteada es el coeficiente de silueta promedio

### Comparación con un gráfico de silueta malo: algoritmo k-meas evaluado para dos centroides

In [ ]:
km = KMeans(n_clusters=2,
            init='k-means++',
            n_init=10,
            max_iter=300,
            tol=1e-04,
            random_state=0)
y_km = km.fit_predict(X)

plt.scatter(X[y_km == 0, 0],
            X[y_km == 0, 1],
            s=50,
            c='lightgreen',
            marker='s',
            label='cluster 1')
plt.scatter(X[y_km == 1, 0],
            X[y_km == 1, 1],
            s=50,
            c='orange',
            marker='o',
            label='cluster 2')

plt.scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
            s=250, marker='*', c='red', label='centroides')
plt.legend()
plt.grid()
plt.tight_layout()
#plt.savefig('./figures/centroids_bad.png', dpi=300)
plt.show()

Uno de los centroides cae entre dos de los 3 agrupamientos
No es completamente terrible pero es insuficiente

In [ ]:
cluster_labels = np.unique(y_km)
n_clusters = cluster_labels.shape[0]
silhouette_vals = silhouette_samples(X, y_km, metric='euclidean')
y_ax_lower, y_ax_upper = 0, 0
yticks = []
for i, c in enumerate(cluster_labels):
    c_silhouette_vals = silhouette_vals[y_km == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(float(i) / n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
             edgecolor='none', color=color)

    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(c_silhouette_vals)
    
silhouette_avg = np.mean(silhouette_vals)
plt.axvline(silhouette_avg, color="red", linestyle="--") 

plt.yticks(yticks, cluster_labels + 1)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')

plt.tight_layout()
# plt.savefig('./figures/silhouette_bad.png', dpi=300)
plt.show()

El grafico tiene diferentes longitudes y anchiaras, esto demuestra un agrupamiento relativamente malo

# Ejemplo 2: clásificación de trilobites

<img src="img/trilobites.jpg" width="30%">

**Intuición**
¿Qué grupos harías con los siguientes datos?,  ¿Cómo sería el proceso?

In [ ]:
X = pd.read_csv("data/cluster1.csv").values+15
plt.scatter(X[:,0], X[:,1])
plt.xlabel("anchura del trilobite")
plt.ylabel("largo del trilobite");

In [ ]:
X = pd.read_csv("data/cluster1.csv").values+15
n_clusters = 2

km = KMeans(n_clusters=n_clusters)
km.fit(X)
y = km.predict(X)

Numero de asignaciones por clase:

In [ ]:
pd.Series(y).value_counts()

Coordenadas de los clusters:

In [ ]:
km.cluster_centers_

In [ ]:
# Ayuda con comando (descomentar)
#help(KMeans)

In [ ]:
cmap = plt.cm.plasma

cmap((y*255./(n_clusters-1)).astype(int))
for i in np.unique(y):
    cmap = plt.cm.bwr
    col = cmap((i*255./(n_clusters-1)).astype(int))
    Xr = X[y==i]
    plt.scatter(Xr[:,0], Xr[:,1], color=col, label="cluster %d"%i, alpha=.5)
plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:,1],marker="x", lw=5, s=200, color="black")
plt.legend()    
plt.xlabel("anchura del trilobite")
plt.ylabel("largo del trilobite");

observa cómo KMeans agrupa datos en 2D con diferentes números de clusters

In [ ]:
X = pd.read_csv("data/cluster1.csv").values
importlib.reload(mlutils)
mlutils.experiment_number_of_clusters(X, KMeans(), show_metric=False)

# Ejemplo 3: Experimenta con distintos datasets sintéticos

- Cambia `cluster_std` y `centers` en `make_blobs` para generar datasets con distintas distribuciones
- Cuál es el númer de clusters _natural_ que usarías? por qué es _natural_?

In [ ]:
X,_ = make_blobs(500, cluster_std=1.5, centers=6)
mlutils.experiment_number_of_clusters(X, KMeans(), show_metric=False)

### ¿Cómo seleccionar el número de clusters?

Consulta <a href="https://en.wikipedia.org/wiki/Silhouette_(clustering)">Silhouette Coefficient</a>

In [ ]:
X = pd.read_csv("data/cluster1.csv").values
mlutils.experiment_number_of_clusters(X, KMeans(), show_metric=True)

¿son **_naturales_** los clusters formados con los siguientes datos?

In [ ]:
X,_ = make_moons(500, noise=.1)
mlutils.plot_cluster_predictions(KMeans(), X, n_clusters=2,cmap=plt.cm.bwr, show_metric=True)

# Ejemplo 4: k-means se limita a los froteras de cluster lineales

Las suposiciones fundamentales del modelo de *k*-means (los puntos estarán más cerca de su propio cluster que de otros) significa que el algoritmo a menudo será ineficaz si los cluster tienen geometrías complicadas.

Las fronteras entre los clusters *k*-means  siempre serán lineales, lo que significa que fallará para límites más complicados.
Considere los siguientes datos, junto con las etiquetas de los cúmulos que se encuentran en el enfoque típico *k*-means:

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(200, noise=.05, random_state=0)

In [ ]:
labels = KMeans(2, random_state=0).fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

Una versión kernelizada de *k*-means se implementa en Scikit-Learn utilizando el estimador de ``SpectralClustering``.
Utiliza el grafo de los vecinos más cercanos para calcular una representación dimensional más alta de los datos, y luego asigna etiquetas utilizando un algoritmo *k*-means:

In [ ]:
from sklearn.cluster import SpectralClustering
model = SpectralClustering(n_clusters=2, affinity='nearest_neighbors',
                           assign_labels='kmeans')
labels = model.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

# Ejemplo 5: compresión de la imagen usando el algoritmo k-mean

<img src="data/tiger.png" width="50%">

In [ ]:
from skimage import io
import numpy as np
import numpy.matlib

In [ ]:
tiger = io.imread('data/tiger.png')
ax = plt.axes(xticks=[], yticks=[])
ax.imshow(tiger);

La imagen en sí se almacena en una matriz tridimensional de tamaño ``(alto, ancho, RGB)``, que contiene contribuciones rojas/azules/verdes como números enteros de 0 a 255:

In [ ]:
tiger.shape

Una forma en que podemos ver este conjunto de píxeles es como una nube de puntos en un espacio de color tridimensional.
Redefiniremos los datos a ``[n_muestras x n_características]``, y reajustaremos los colores para que estén entre 0 y 1:

In [ ]:
rows = tiger.shape[0]
cols = tiger.shape[1]
tiger = tiger/255.0
data = tiger.reshape(tiger.shape[0]*tiger.shape[1],3) #693 * 1232 * 3
data.shape

Podemos visualizar estos píxeles en este espacio de color, usando un subconjunto de 10.000 píxeles para la eficiencia:

In [ ]:
def plot_pixels(data, title, colors=None, N=10000):
    if colors is None:
        colors = data
    
    # choose a random subset
    rng = np.random.RandomState(0)
    i = rng.permutation(data.shape[0])[:N]
    colors = colors[i]
    R, G, B = data[i].T
    
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    ax[0].scatter(R, G, color=colors, marker='.')
    ax[0].set(xlabel='Red', ylabel='Green', xlim=(0, 1), ylim=(0, 1))

    ax[1].scatter(R, B, color=colors, marker='.')
    ax[1].set(xlabel='Red', ylabel='Blue', xlim=(0, 1), ylim=(0, 1))

    fig.suptitle(title, size=20);

In [ ]:
plot_pixels(data, title='Espacio de color de entrada: 16 millones de colores posibles')

Ahora vamos a reducir estos 16 millones de colores a sólo 16 colores, usando una agrupación *k*-means a través del espacio de píxeles.
Debido a que estamos tratando con un conjunto de datos muy grande, usaremos el mini lote *(mini batch)* *k*-means, que opera en subconjuntos de datos para calcular el resultado mucho más rápido que el algoritmo estándar *k*-means:

In [ ]:
import warnings; warnings.simplefilter('ignore')  # Fix NumPy issues.

from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(16)
kmeans.fit(data)
new_colors = kmeans.cluster_centers_[kmeans.predict(data)]

plot_pixels(data, colors=new_colors,
            title="Espacio de color reducido: 16 colores")

El resultado es un re-coloreado de los píxeles originales, donde a cada píxel se le asigna el color de su centro de cúmulo más cercano.
Mostrar estos nuevos colores en el espacio de la imagen en lugar del espacio de los píxeles nos muestra el efecto:

In [ ]:
tiger_recolored = new_colors.reshape(tiger.shape)

fig, ax = plt.subplots(1, 2, figsize=(16, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)
ax[0].imshow(tiger)
ax[0].set_title('Imagen original', size=16)
ax[1].imshow(tiger_recolored)
ax[1].set_title('Imagen de 16-colores', size=16);

* Algunos detalles se pierden enla derecha, pero la imagen general sigue siendo fácilmente reconocible.
* ¡Esta imagen de la derecha alcanza un factor de compresión de alrededor de 1 millón!

In [ ]:
import imageio
imageio.imwrite('tiger_recolored.png', tiger_recolored)

In [ ]:
image_compressed = io.imread('tiger_recolored.png')
io.imshow(image_compressed)
io.show()

A continuación de puede apreciar la diferencia entre los pesos de los archivos

In [ ]:
import os
info = os.stat('data/tiger.png')
print("tamaño de la imagen antes de ejecutar el algoritmo k-mean: ",info.st_size/1024,"KB")
info = os.stat('tiger_recolored.png')
print("tamaño de la imagen despues de ejecutar el algoritmo k-mean: ",info.st_size/1024,"KB")

## Referencias generales

- [Cluster Analysis on Wikipedia](https://en.wikipedia.org/wiki/Cluster_analysis)
- [Cluster Analysis, Basic concepts and algorithms](https://www-users.cs.umn.edu/~kumar001/dmbook/ch8.pdf)